In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

In [3]:
# SUUMOのURL
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC011/?ar=030&bs=040&fw=%E5%8D%83%E4%BB%A3%E7%94%B0%E5%8C%BA"

In [4]:
# リクエストを送信
response = requests.get(url)

# BeautifulSoupオブジェクトを作成
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# 物件のリストを初期化
properties_list = []

# すべての物件リストアイテムを取得
property_items = soup.find_all('div', class_='property')

# 物件が見つかった場合の処理
for item in property_items:
    property_data = {
        '物件名': item.find('a', class_='js-cassetLinkHref').text if item.find('a', class_='js-cassetLinkHref') else '-',
        'URL': 'https://suumo.jp' + item.find('a', class_='js-cassetLinkHref')['href'] if item.find('a', class_='js-cassetLinkHref') else '-',
        'イメージ画像': item.find('img', class_='js-noContextMenu')['rel'] if item.find('img', class_='js-noContextMenu') else '-',
        '賃料': int(float(item.find('div', class_='detailbox-property-point').text.replace('万円', '')) * 10000) if item.find('div', class_='detailbox-property-point') else '-',
        '管理費': item.find('td', class_='detailbox-property--col1').find_all('div')[1].text.replace('管理費 ', '') if item.find('td', class_='detailbox-property--col1') and len(item.find('td', class_='detailbox-property--col1').find_all('div')) > 1 else '-',
        '間取り': item.find('td', class_='detailbox-property--col3').find_all('div')[0].text if item.find('td', class_='detailbox-property--col3') else '-',
        '専有面積': item.find('td', class_='detailbox-property--col3').find_all('div')[1].text.replace('\n', '').strip() if item.find('td', class_='detailbox-property--col3') else '-',
        '向き': item.select('td.detailbox-property--col3')[0].find_all('div')[2].text if len(item.select('td.detailbox-property--col3')[0].find_all('div')) > 2 else '-' if item.select('td.detailbox-property--col3') else '-',
        '物件種別': item.select('td.detailbox-property--col3')[1].find_all('div')[0].text if len(item.select('td.detailbox-property--col3')) > 1 else '-' if item.select('td.detailbox-property--col3') else '-',
        '築年数': item.select('td.detailbox-property--col3')[1].find_all('div')[1].text if len(item.select('td.detailbox-property--col3')[1].find_all('div')) > 1 else '-' if item.select('td.detailbox-property--col3') else '-',
        '住所': item.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])').text.strip() if item.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])') else '-',
        '最寄り駅と距離': item.select_one('.detailnote-box > div').text.strip() if item.select_one('.detailnote-box > div') else '-',
        '敷金': item.find('td', class_='detailbox-property--col2').find_all('div')[0].contents[-1].strip() if item.find('td', class_='detailbox-property--col2') else '-',
        '礼金': item.find('td', class_='detailbox-property--col2').find_all('div')[1].contents[-1].strip() if item.find('td', class_='detailbox-property--col2') else '-',
        '保証金': item.find('div', class_='detailbox-property-inactive', string=lambda text: '保証金' in text).text.replace('保証金', '').strip() if item.find('div', class_='detailbox-property-inactive', string=lambda text: '保証金' in text) else '-',
        '敷引・償却': item.find('div', class_='detailbox-property-inactive', string=lambda text: '敷引・償却' in text).text.replace('敷引・償却', '').strip() if item.find('div', class_='detailbox-property-inactive', string=lambda text: '敷引・償却' in text) else '-',
        '取り扱い店舗': item.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)').text.strip() if item.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)') else '-',
        '取り扱い店舗の電話番号': item.find('div', class_='ui-text--bold').text.strip() if item.find('div', class_='ui-text--bold') else '-'
    }

    properties_list.append(property_data)

# データフレームに変換
df = pd.DataFrame(properties_list)

# データフレームを表示
print(df)

# データフレームをCSVファイルに出力
df.to_csv('chiyoda_page1_all_property.csv', index=False, encoding='utf-8-sig')
print("データがCSVファイル 'chiyoda_page1_all_property.csv' に保存されました。")


                 物件名                                        URL  \
0        イシカワビレッジ 3階  https://suumo.jp/chintai/bc_100192305263/   
1         メゾンホワイト 2階  https://suumo.jp/chintai/bc_100371535158/   
2         鼻高フラットA 1階  https://suumo.jp/chintai/bc_100371535133/   
3     アシスト城南マンション 5階  https://suumo.jp/chintai/bc_100379476278/   
4     スプリングフィールドI 1階  https://suumo.jp/chintai/bc_100365391761/   
5     ソレイユ若葉セブンツー 2階  https://suumo.jp/chintai/bc_100356944625/   
6       トランビュー沼田A 2階  https://suumo.jp/chintai/bc_100374027144/   
7           ハイム三田 2階  https://suumo.jp/chintai/bc_100297643799/   
8      志村坂上 ワンルーム 1階  https://suumo.jp/chintai/bc_100376496541/   
9       LIFE　BASE 2階  https://suumo.jp/chintai/bc_100371535221/   
10         シナーラ衣笠 1階  https://suumo.jp/chintai/bc_100371535145/   
11       ベルヴュー高崎B 1階  https://suumo.jp/chintai/bc_100378635090/   
12  ワコーレエレガンス行田第2 2階  https://suumo.jp/chintai/bc_100378815468/   
13            ダリア 1階  https://suumo.jp/chintai/bc_100378815550

In [6]:
import os

# 現在の作業ディレクトリを取得
current_directory = os.getcwd()
print("現在の作業ディレクトリ:", current_directory)

# 物件のリストを初期化
properties_list = []

# すべての物件リストアイテムを取得
property_items = soup.find_all('div', class_='property')

for item in property_items:
    property_data = {
        '物件名': item.find('a', class_='js-cassetLinkHref').text if item.find('a', class_='js-cassetLinkHref') else '-',
        'URL': 'https://suumo.jp' + item.find('a', class_='js-cassetLinkHref')['href'] if item.find('a', class_='js-cassetLinkHref') else '-',
        'イメージ画像': item.find('img', class_='js-noContextMenu')['rel'] if item.find('img', class_='js-noContextMenu') else '-',
        '賃料': int(float(item.find('div', class_='detailbox-property-point').text.replace('万円', '')) * 10000) if item.find('div', class_='detailbox-property-point') else '-',
        '管理費': item.find('td', class_='detailbox-property--col1').find_all('div')[1].text.replace('管理費 ', '') if item.find('td', class_='detailbox-property--col1') and len(item.find('td', class_='detailbox-property--col1').find_all('div')) > 1 else '-',
        '間取り': item.find('td', class_='detailbox-property--col3').find_all('div')[0].text if item.find('td', class_='detailbox-property--col3') else '-',
        '専有面積': item.find('td', class_='detailbox-property--col3').find_all('div')[1].text.replace('\n', '').strip() if item.find('td', class_='detailbox-property--col3') else '-',
        '向き': item.select('td.detailbox-property--col3')[0].find_all('div')[2].text if len(item.select('td.detailbox-property--col3')[0].find_all('div')) > 2 else '-' if item.select('td.detailbox-property--col3') else '-',
        '物件種別': item.select('td.detailbox-property--col3')[1].find_all('div')[0].text if len(item.select('td.detailbox-property--col3')) > 1 else '-' if item.select('td.detailbox-property--col3') else '-',
        '築年数': item.select('td.detailbox-property--col3')[1].find_all('div')[1].text if len(item.select('td.detailbox-property--col3')[1].find_all('div')) > 1 else '-' if item.select('td.detailbox-property--col3') else '-',
        '住所': item.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])').text.strip() if item.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])') else '-',
        '最寄り駅と距離': item.select_one('.detailnote-box > div').text.strip() if item.select_one('.detailnote-box > div') else '-',
        '敷金': item.find('td', class_='detailbox-property--col2').find_all('div')[0].contents[-1].strip() if item.find('td', class_='detailbox-property--col2') else '-',
        '礼金': item.find('td', class_='detailbox-property--col2').find_all('div')[1].contents[-1].strip() if item.find('td', class_='detailbox-property--col2') else '-',
        '保証金': item.find('div', class_='detailbox-property-inactive', string=lambda text: '保証金' in text).text.replace('保証金', '').strip() if item.find('div', class_='detailbox-property-inactive', string=lambda text: '保証金' in text) else '-',
        '敷引・償却': item.find('div', class_='detailbox-property-inactive', string=lambda text: '敷引・償却' in text).text.replace('敷引・償却', '').strip() if item.find('div', class_='detailbox-property-inactive', string=lambda text: '敷引・償却' in text) else '-',
        '取り扱い店舗': item.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)').text.strip() if item.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)') else '-',
        '取り扱い店舗の電話番号': item.find('div', class_='ui-text--bold').text.strip() if item.find('div', class_='ui-text--bold') else '-'
    }

    properties_list.append(property_data)

# データフレームに変換
df = pd.DataFrame(properties_list)

# データフレームをCSVファイルに出力
df.to_csv('chiyoda_page1_all_property.csv', index=False, encoding='utf-8-sig')
print("データがCSVファイル 'chiyoda_page1_all_property.csv' に保存されました。")

# SQLiteデータベースのパスを設定（現在の作業ディレクトリに保存）
database_path = os.path.join(current_directory, 'chiyoda_page1_all_property.sqlite')

# SQLiteデータベースに接続（なければ新規作成）
conn = sqlite3.connect('chiyoda_page1_all_property.sqlite')
c = conn.cursor()

# テーブルを作成
c.execute('''
CREATE TABLE IF NOT EXISTS properties (
    物件名 TEXT,
    URL TEXT,
    イメージ画像 TEXT,
    賃料 INTEGER,
    管理費 TEXT,
    間取り TEXT,
    専有面積 TEXT,
    向き TEXT,
    物件種別 TEXT,
    築年数 TEXT,
    住所 TEXT,
    最寄り駅と距離 TEXT,
    敷金 TEXT,
    礼金 TEXT,
    保証金 TEXT,
    敷引・償却 TEXT,
    取り扱い店舗 TEXT,
    取り扱い店舗の電話番号 TEXT
)
''')

# データフレームの内容をSQLiteに挿入
df.to_sql('properties', conn, if_exists='replace', index=False)

# コネクションを閉じる
conn.close()
print(f"データがSQLiteデータベース '{database_path}' に保存されました。")

現在の作業ディレクトリ: c:\Users\jiebing\Desktop\ChintaiAPP\make_db
データがCSVファイル 'chiyoda_page1_all_property.csv' に保存されました。
データがSQLiteデータベース 'c:\Users\jiebing\Desktop\ChintaiAPP\make_db\chiyoda_page1_all_property.sqlite' に保存されました。
